Code to predict type of pmeumonia from exray of chest of patient.

In [8]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from os import listdir
from os.path import isfile, join
import numpy as np
import tensorflow as tf
from PIL import Image as img
from tensorflow.python import keras
from keras import Sequential, datasets, layers, models
import pickle as pk

Various imports needed.

In [2]:
pixels = 150
dimensions = (pixels, pixels)
bins = 2
mid_layer = 750
training_iters = 10
learning_rate = 2e-3
num_epochs = 1
reduction_factor = 0.9

Variables defined

In [3]:
class LearningRateReducerCb(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        old_lr = self.model.optimizer.lr.read_value()
        new_lr = old_lr * reduction_factor
        print("\nEpoch: {}. Reducing Learning Rate from {} to {}".format(epoch, old_lr, new_lr))
        self.model.optimizer.lr.assign(new_lr)

Reduces learning rate by epochs.

In [4]:
train_images = []
train_labels = np.array([])
folder_path = "/mnt/c/NN/Pneumonia/chest_xray/train/"

for f in listdir(folder_path + "NORMAL"):
    im = img.open(folder_path + "NORMAL/" + f)
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    train_images.append(im_array)
    im.close()
    train_labels = np.append(train_labels, 0)

for f in listdir(folder_path + "PNEUMONIA"):
    im = img.open(folder_path + "PNEUMONIA/" + f)
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    train_images.append(im_array)
    im.close()

    train_labels = np.append(train_labels, 1)    
    # if "bacteria" in f:
        # train_labels = np.append(train_labels, 1)
    # elif "virus" in f:
        # train_labels = np.append(train_labels, 2)

test_images = []
test_labels = np.array([])
folder_path = "/mnt/c/NN/Pneumonia/chest_xray/test/"

for f in listdir(folder_path + "NORMAL"):
    im = img.open(folder_path + "NORMAL/" + f)
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    test_images.append(im_array)
    im.close()    
    test_labels = np.append(test_labels, 0)

for f in listdir(folder_path + "PNEUMONIA"):
    im = img.open(folder_path + "PNEUMONIA/" + f)
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    test_images.append(im_array)
    im.close()

    test_labels = np.append(test_labels, 1)    
    # if "bacteria" in f:
        # test_labels = np.append(test_labels, 1)
    # elif "virus" in f:
        # test_labels = np.append(test_labels, 2)

train_images = np.array(train_images)
test_images = np.array(test_images)

train_images = train_images / 255.0
test_images = test_images / 255.0

Images Imported
Average time taken = 150 seconds.

In [5]:
model = models.Sequential()
model.add(layers.Conv2D(pixels, (3, 3), activation='relu', input_shape=(pixels, pixels, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(2 * pixels, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(2 * pixels, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(pixels, activation='relu'))
model.add(layers.Dense(bins))

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Creates CNN model

In [6]:
history = model.fit(train_images, train_labels, callbacks=[LearningRateReducerCb()], epochs=num_epochs,
                    validation_data=(test_images, test_labels))

163/163 [==============================] - ETA: 0s - loss: 0.4933 - accuracy: 0.8424
Epoch: 0. Reducing Learning Rate from 0.0020000000949949026 to 0.0018000000854954123
163/163 [==============================] - 613s 4s/step - loss: 0.4933 - accuracy: 0.8424 - val_loss: 0.8780 - val_accuracy: 0.7324


Trains the model.
Number of epochs = 3
Training time = 33:27

In [7]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print("Accuracy = " + str(np.round(test_acc * 100, 3)) + " %")

20/20 - 20s - loss: 0.8780 - accuracy: 0.7324 - 20s/epoch - 1s/step
Accuracy = 73.237 %


Prints accuracy for the model.
Validation Accuracy reached = 65

In [10]:
filename = 'model.sav'
pk.dump(model, open(filename, 'wb'))

INFO:tensorflow:Assets written to: ram://eb996c35-3c90-4453-84b2-939ab3323a05/assets


In [22]:
filepath = "/mnt/c/NN/Pneumonia/chest_xray/train/NORMAL/IM-0115-0001.jpeg"
test_image = img.open(filepath)
test_image = test_image.resize(dimensions)
test_image = np.asarray(test_image)
test_image = np.reshape(test_image, (pixels, pixels, 1))
test_image = np.expand_dims(test_image, 0)
print(np.shape(test_image))
print(model.predict(test_image))

(1, 150, 150, 1)


AttributeError: 'Sequential' object has no attribute 'predict_class'